# Data Gathering

In [ ]:
%matplotlib widget

In [ ]:
from utils.io import gdf_viewer, gdf_merger, data_validation, gdf_filter, fix_duplicates
import re, os
import numpy as np
import geopandas as gpd
import pandas as pd
import datetime as dtm
import matplotlib.pyplot as plt
from definitions import ROOT_DIR

In [ ]:
def files_search(work_dir, files_dict, prefix='', skip=None, details=False):   
    
    if skip is None:
        skip = "we don't want to skip a word"
        
    for k in files_dict.keys():
        tmp_list = []
        for p,d,f in os.walk(work_dir):
            for x in f:
                add = False
                if re.search(prefix,x,re.I) and not re.search(skip,x,re.I):
                    add = True
                    i = str(f'{p}/{x}')
                else:
                    add = False
                    i=''
                    
                if re.search(k,i,re.I) and add:
                    tmp_list.append(i)
        tmp_list.sort()
        files_dict.update({k:tmp_list})

    for k,v in files_dict.items():
        print(k,' \t: ',len(v))
    
    if details: # Look filenames
        which = files_dict.keys()

        for w in which:
            print('\n+++++++++++++++++')
            print(f'+  {w.upper()}\t+ ')
            print('+++++++++++++++++')
            [print(i, '-', x) for i, x in enumerate(files_dict[w], 0)]    


In [ ]:
def create_df(files, verbose=True): # find another name for this function
    """
    create dataframes from files and test if they contain position informations
    files: list of files name
    """
    dfs = []
    for f in files:
        df = pd.read_csv(f, delimiter=',')
        dfs.append(df)
        
        if verbose:
            if 'X' in list(df.columns): msg = ' --> Coordinates'
            else: msg = ' --> No coordinates'

            print(f"df1 : {msg}")
            
    return dfs

In [ ]:
def dataset_overview(d, verbose=False): # check for same datasets in given files
    """d: dict
    """
    l = len(d)
    with_coord = []
    no_coord = []
    same = []
    
    for i in range(l-1):
        for j in range(i,l):
            a, b = create_df([d[i], d[j]], verbose)
            if j != i:
                if a.equals(b):
                    same.append((i,j))
            
            if 'X' in list(b.columns) and j not in with_coord:
                with_coord.append(j)
            elif 'X' not in list(b.columns) and j not in no_coord:
                no_coord.append(j)
    
    print(f'Same files:{same}\nFiles with coordinates:{with_coord}\nFiles without coordinates:{no_coord}')

## Reading files

In [ ]:
work_dir = ROOT_DIR+'/CF_data/Result_traitem/'
save_dir = ROOT_DIR+'/CF_data/Donnees_fusionnees/'

In [ ]:
# create my dictionary structure to retrieve good files (Keynames !!!)
files_dict={'Borehole':0,'Piezometer':0,'Piezair':0,'Trench':0,'Litho':0,'Equipm':0,
        'Measure':0,'Sample':0,'Analysis':0,'Facility':0}

In [ ]:
files_search(work_dir, files_dict, prefix='', skip='source')

In [ ]:
how=['inner', 'outer', 'left', 'right']

In [ ]:
f = False
t = True

# ================== PROCESSING ===================== 

# Boreholes

Some corrections todo in 'data organization':
- correct extraction in the file 2 -> Samples
- file 4 and file 5 are the same in result (check it)
- try to concatenate file 1 with piezo (if possible because no position)
- check processing for 'refus and 'type_refus' (every object)

In [ ]:
key='Borehole'
save_file = f'Merged_Boreholes.csv'
coi=['ID','ID_date','X','Y','Z','Type','Long_for','Diam_for','Refus', 'Societe'] #columns of interest
dataset = pd.DataFrame() # for saving object info after last merging
print(len(files_dict[key]), 'files')

In [ ]:
files_dict[key] #files_dict[key][0]

In [ ]:
dataset_overview(files_dict[key])

#### $\color{green}{\textbf{Read and merge}}$

In [ ]:
file1= work_dir + 'profils_sols_donnees_forages/Log_Boreholes.csv' # 5
file2= work_dir + 'profils_sols_donnees_forages/Equipement_Boreholes.csv' # 4

df1, df2 = create_df([file1, file2])
gdf_viewer(df1, rows=3, un_val='ID', view=t), gdf_viewer(df2, rows=3, un_val='ID', view=t)

In [ ]:
df1.rename(columns={'Profondeur':'Long_for'}, inplace=True)
df2.rename(columns={'Profondeur':'Long_for'}, inplace=True)

In [ ]:
mdf, conflict_df=gdf_merger(df1, df2, how=how[1], on='ID', dist_max=1)

In [ ]:
gdf_viewer(mdf, rows=10, un_val='ID', view=t)

#### First object dataset save

In [ ]:
dataset = mdf.copy() #saving

#### $\color{green}{\textbf{Read and merge}}$

In [ ]:
file1= work_dir + 'database_Memoris3/Profils_sol_Boreholes.csv' # 2
file2= work_dir + 'Prof_contact_sol_forage/Feuil1_Boreholes.csv' # 1

df1, df2 = create_df([file1, file2])
gdf_viewer(df1, rows=3, un_val='ID', view=t), gdf_viewer(df2, rows=3, un_val='ID', view=t)

In [ ]:
df1.rename(columns={'Profondeur':'Long_for'}, inplace=True)
df2.rename(columns={'Profondeur':'Long_for'}, inplace=True)

In [ ]:
mdf, conflict_df=gdf_merger(df1, df2, how=how[1], on='ID', dist_max=1)

In [ ]:
gdf_viewer(mdf, rows=10, un_val='ID', view=t)

#### $\color{blue}{\textbf{Manage conflicts}}$

In [ ]:
conflict_df

#### Merge with object dataset

In [ ]:
dataset, conflict_df=gdf_merger(dataset, mdf, how=how[1], on='ID', dist_max=1)

In [ ]:
if 'level_0' in dataset.columns:
    if 'index' in dataset.columns:
        dataset.drop(columns='index', inplace=True)
    dataset.rename(columns={'level_0':'index'}, inplace=True)

In [ ]:
gdf_viewer(dataset, rows=10, un_val='ID', view=t)

#### $\color{green}{\textbf{Read and merge}}$

In [ ]:
file1= work_dir + 'Forage_Pilote/leve_Z_elect_pos_Boreholes.csv' # 0
file2= work_dir + 'donnees_terrain_2019/Donnees_forage_Boreholes.csv' # 3

df1, df2 = create_df([file1, file2])
gdf_viewer(df1, rows=3, un_val='ID', view=t), gdf_viewer(df2, rows=3, un_val='ID', view=t)

In [ ]:
df2.ID=df2.ID.apply(lambda x: 'F'+x) # name recent (2019) boreholes

In [ ]:
mdf, conflict_df=gdf_merger(df1, df2, how=how[1], on='ID', dist_max=1)

In [ ]:
gdf_viewer(mdf, rows=10, un_val='ID', view=t)

#### $\color{blue}{\textbf{Manage conflicts}}$

In [ ]:
conflict_df

#### Merge with object dataset

In [ ]:
dataset, conflict_df=gdf_merger(dataset, mdf, how=how[1], on='ID', dist_max=1)

In [ ]:
if 'level_0' in dataset.columns:
    if 'index' in dataset.columns:
        dataset.drop(columns='index', inplace=True)
    dataset.rename(columns={'level_0':'index'}, inplace=True)

In [ ]:
gdf_viewer(dataset, rows=10, un_val='ID', view=t)

#### $\color{blue}{\textbf{Manage conflicts}}$

In [ ]:
conflict_df

In [ ]:
gdf_viewer(dataset, rows=3, un_val='ID', view=t)

####  $\color{red}{\textbf{Save final object dataset}}$

In [ ]:
if 'index' in dataset.columns:
    dataset.drop(columns='index', inplace=True)

In [ ]:
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
    
dataset.to_csv(save_dir + save_file, index=False)

# Unknown facilities

In [ ]:
key='Facility'
save_file = f'Merged_Facilites_unknw.csv'
#coi=['ID','X','Y','Z','Litho_top','Litho_base','Description']  #columns of interest
dataset = pd.DataFrame()
print(len(files_dict[key]), 'files')

In [ ]:
files_dict[key]

In [ ]:
dataset_overview(files_dict[key])

#### $\color{green}{\textbf{Read and merge}}$

In [ ]:
file1= work_dir + 'database_Memoris3/Donnees_piezos_Unkown-facility.csv' # 0
file2= work_dir + 'database_Memoris3/Drains_Pz_ENEL_Unkown-facility.csv' # 1  

df1, df2 = create_df([file1, file2])
gdf_viewer(df1, rows=3, un_val='ID', view=t), gdf_viewer(df2, rows=3, un_val='ID', view=t)

In [ ]:
mdf, conflict_df=gdf_merger(df1, df2, how=how[1], on='ID', dist_max=1)

#### First object dataset save

In [ ]:
dataset = mdf.copy() #saving

In [ ]:
gdf_viewer(dataset, rows=3, un_val='ID', view=t)

#### $\color{green}{\textbf{Read and merge}}$

In [ ]:
file1= work_dir + 'database_Memoris3/Result_eau_Unkown-facility.csv' # 2

df1 = create_df([file1])[0]
gdf_viewer(df1, rows=3, un_val='ID', view=t)

#### Merge with object dataset

In [ ]:
dataset, conflict_df=gdf_merger(dataset, df1, how=how[1], on='ID', dist_max=1)

In [ ]:
if 'level_0' in dataset.columns:
    if 'index' in dataset.columns:
        dataset.drop(columns='index', inplace=True)
    dataset.rename(columns={'level_0':'index'}, inplace=True)

In [ ]:
gdf_viewer(dataset, rows=10, un_val='ID', view=t)

####  $\color{red}{\textbf{Save final object dataset}}$

In [ ]:
if 'index' in dataset.columns:
    dataset.drop(columns='index', inplace=True)

In [ ]:
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
    
dataset.to_csv(save_dir + save_file, index=False)

# Equipments

In [ ]:
key='Equipm'
save_file = f'Merged_Equipments.csv'
coi=['ID','ID_date','X','Y','Z','Type','Long_for','Diam_for','Refus'] #columns of interest
dataset = pd.DataFrame()
print(len(files_dict[key]), 'files')

In [ ]:
files_dict[key]

In [ ]:
dataset_overview(files_dict[key])

#### $\color{green}{\textbf{Read and merge}}$

In [ ]:
file1= work_dir + 'donnees_terrain_2019/Equipement_Equipment.csv' # 1
file2= work_dir + 'profils_sols_donnees_forages/Equipement_Equipments.csv' # 2  


df1, df2 = create_df([file1, file2])
gdf_viewer(df1, rows=3, un_val='ID', view=t), gdf_viewer(df2, rows=3, un_val='ID', view=t)

In [ ]:
df1.ID = df1.ID.astype(str)
df2.ID = df2.ID.astype(str)

In [ ]:
mdf, conflict_df=gdf_merger(df1, df2, how=how[1], on='ID', dist_max=1)

#### First object dataset save

In [ ]:
dataset = mdf.copy() #saving

In [ ]:
gdf_viewer(dataset, rows=3, un_val='ID', view=t)

####  $\color{red}{\textbf{Save final object dataset}}$

In [ ]:
if 'index' in dataset.columns:
    dataset.drop(columns='index', inplace=True)

In [ ]:
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
    
dataset.to_csv(save_dir + save_file, index=False)

# Samples

In [ ]:
key='Sample'
save_file = f'Merged_Samples.csv'
coi=['ID','ID_date','X','Y','Z','Type','Long_for','Diam_for','Refus'] #columns of interest
dataset = pd.DataFrame()
print(len(files_dict[key]), 'files')

In [ ]:
files_dict[key]

In [ ]:
dataset_overview(files_dict[key])

#### $\color{green}{\textbf{Read and merge}}$

In [ ]:
file1= work_dir + 'Memoris_seafile/Result_eau_Samples-water.csv' # 4
file2= work_dir + 'Phase_1_Memoris/Result_eau_Samples-water.csv' # 5

df1, df2 = create_df([file1, file2])
gdf_viewer(df1, rows=3, un_val='ID_ech', view=t), gdf_viewer(df2, rows=3, un_val='ID_ech', view=t)

In [ ]:
mdf, conflict_df=gdf_merger(df1, df2, how=how[1], on='ID_ech', date_col='Date_prv', dist_max=1)

#### First object dataset save

In [ ]:
dataset = mdf.copy() #saving

In [ ]:
gdf_viewer(dataset, rows=3, un_val='ID_ech', view=t)

#### $\color{green}{\textbf{Read and merge}}$

In [ ]:
file1= work_dir + 'vUmons_logsFor/Analyse_eau_Phases1&2_Samples-water.csv' # 25
file2= work_dir + 'vUmons_logsFor/Analyse_sol_Phases1&2_Samples-soil.csv' # 26


df1, df2 = create_df([file1, file2])
gdf_viewer(df1, rows=3, un_val='ID_ech', view=t), gdf_viewer(df2, rows=3, un_val='ID_ech', view=t)

In [ ]:
mdf, conflict_df=gdf_merger(df1, df2, how=how[1], on='ID_ech', date_col='Date_prv', dist_max=1)

#### Merge with object dataset

In [ ]:
dataset, conflict_df=gdf_merger(dataset, mdf, how=how[1], on='ID_ech', date_col='Date_prv', dist_max=1)

In [ ]:
if 'level_0' in dataset.columns:
    if 'index' in dataset.columns:
        dataset.drop(columns='index', inplace=True)
    dataset.rename(columns={'level_0':'index'}, inplace=True)

In [ ]:
gdf_viewer(dataset, rows=10, un_val='ID_ech', view=t)

#### $\color{green}{\textbf{Read and merge}}$

In [ ]:
file1= work_dir + 'Container_phyto/Param_agro_Samples-soil.csv' # 0
file2= work_dir + 'Phase_2_Memoris/Result_eau_Samples-water.csv' # 8


df1, df2 = create_df([file1, file2])
gdf_viewer(df1, rows=3, un_val='ID_ech', view=t), gdf_viewer(df2, rows=3, un_val='ID_ech', view=t)

In [ ]:
mdf, conflict_df=gdf_merger(df1, df2, how=how[1], on='ID_ech', date_col='Date_prv', dist_max=1)

#### Merge with object dataset

In [ ]:
dataset, conflict_df=gdf_merger(dataset, mdf, how=how[1], on='ID_ech', date_col='Date_prv', dist_max=1)

In [ ]:
if 'level_0' in dataset.columns:
    if 'index' in dataset.columns:
        dataset.drop(columns='index', inplace=True)
    dataset.rename(columns={'level_0':'index'}, inplace=True)

In [ ]:
gdf_viewer(dataset, rows=10, un_val='ID_ech', view=t)

#### $\color{green}{\textbf{Read and merge}}$

In [ ]:
file1= work_dir + 'Container_phyto/Param_agro_Samples-water.csv' # 1
file2= work_dir + 'Container_phyto/Result_SOL_Samples-soil.csv' # 2

df1, df2 = create_df([file1, file2])
gdf_viewer(df1, rows=3, un_val='ID_ech', view=t), gdf_viewer(df2, rows=3, un_val='ID_ech', view=t)

In [ ]:
df1.ID_ech = df1.ID_ech.astype(str)
df2.ID_ech = df2.ID_ech.astype(str)

In [ ]:
mdf, conflict_df=gdf_merger(df1, df2, how=how[1], on='ID_ech', date_col='Date_prv', dist_max=1)

#### Merge with object dataset

In [ ]:
dataset, conflict_df=gdf_merger(dataset, mdf, how=how[1], on='ID_ech', date_col='Date_prv', dist_max=1)

In [ ]:
if 'level_0' in dataset.columns:
    if 'index' in dataset.columns:
        dataset.drop(columns='index', inplace=True)
    dataset.rename(columns={'level_0':'index'}, inplace=True)

In [ ]:
gdf_viewer(dataset, rows=10, un_val='ID_ech', view=t)

#### $\color{blue}{\textbf{Manage conflicts}}$

In [ ]:
conflict_df

#### $\color{green}{\textbf{Read and merge}}$

In [ ]:
file1= work_dir + 'Memoris_seafile/Result_SOL_Samples-soil.csv' # 3
file2= work_dir + 'Phase_1_Memoris/Result_sol_Samples-soil.csv' # 6

df1, df2 = create_df([file1, file2])
gdf_viewer(df1, rows=3, un_val='ID_ech', view=t), gdf_viewer(df2, rows=3, un_val='ID_ech', view=t)

In [ ]:
mdf, conflict_df=gdf_merger(df1, df2, how=how[1], on='ID_ech', date_col='Date_prv', dist_max=1)

#### Merge with object dataset

In [ ]:
dataset, conflict_df=gdf_merger(dataset, mdf, how=how[1], on='ID_ech', date_col='Date_prv', dist_max=1)

In [ ]:
if 'level_0' in dataset.columns:
    if 'index' in dataset.columns:
        dataset.drop(columns='index', inplace=True)
    dataset.rename(columns={'level_0':'index'}, inplace=True)

In [ ]:
gdf_viewer(dataset, rows=10, un_val='ID_ech', view=t)

#### $\color{blue}{\textbf{Manage conflicts}}$

In [ ]:
conflict_df

#### $\color{green}{\textbf{Read and merge}}$

In [ ]:
file1= work_dir + 'Phase_2_Memoris/Result_SOL_Samples-soil.csv' # 7
file2= work_dir + 'Siterem_Ext_Pilote/Param_physico_Samples-water.csv' # 10

df1, df2 = create_df([file1, file2])
gdf_viewer(df1, rows=3, un_val='ID_ech', view=t), gdf_viewer(df2, rows=3, un_val='ID_ech', view=t)

In [ ]:
df1.ID_ech = df1.ID_ech.astype(str)
df2.ID_ech = df2.ID_ech.astype(str)

In [ ]:
mdf, conflict_df=gdf_merger(df1, df2, how=how[1], on='ID_ech', date_col='Date_prv', dist_max=1)

#### Merge with object dataset

In [ ]:
dataset, conflict_df=gdf_merger(dataset, mdf, how=how[1], on='ID_ech', date_col='Date_prv', dist_max=1)

In [ ]:
if 'level_0' in dataset.columns:
    if 'index' in dataset.columns:
        dataset.drop(columns='index', inplace=True)
    dataset.rename(columns={'level_0':'index'}, inplace=True)

In [ ]:
gdf_viewer(dataset, rows=10, un_val='ID_ech', view=t)

#### $\color{blue}{\textbf{Manage conflicts}}$

In [ ]:
conflict_df

#### $\color{green}{\textbf{Read and merge}}$

In [ ]:
file1= work_dir + 'Siterem_Ext_Pilote/Result_eau_Samples-water.csv' # 11
file2= work_dir + 'Siterem_Pilote/Inorganic_major_Samples-water.csv' # 12

df1, df2 = create_df([file1, file2])
gdf_viewer(df1, rows=3, un_val='ID_ech', view=t), gdf_viewer(df2, rows=3, un_val='ID_ech', view=t)

In [ ]:
df1.ID_ech = df1.ID_ech.astype(str)
df2.ID_ech = df2.ID_ech.astype(str)

In [ ]:
mdf, conflict_df=gdf_merger(df1, df2, how=how[1], on='ID_ech', date_col='Date_prv', dist_max=1)

#### Merge with object dataset

In [ ]:
dataset, conflict_df=gdf_merger(dataset, mdf, how=how[1], on='ID_ech', date_col='Date_prv', dist_max=1)

In [ ]:
if 'level_0' in dataset.columns:
    if 'index' in dataset.columns:
        dataset.drop(columns='index', inplace=True)
    dataset.rename(columns={'level_0':'index'}, inplace=True)

In [ ]:
gdf_viewer(dataset, rows=10, un_val='ID_ech', view=t)

#### $\color{blue}{\textbf{Manage conflicts}}$

In [ ]:
conflict_df

In [ ]:
a, b = 4, 0
file1= files_dict[key][a]
file2= files_dict[key][b]
print(files_dict[key][a].replace(work_dir,""),'||', files_dict[key][b].replace(work_dir,""))

In [ ]:
pause

#### $\color{green}{\textbf{Read and merge}}$

In [ ]:
#file1= work_dir + 'Memoris_seafile/Result_SOL_Samples-soil.csv' # 3
#file2= work_dir + 'Phase_1_Memoris/Result_sol_Samples-soil.csv' # 6

df1, df2 = create_df([file1, file2])
gdf_viewer(df1, rows=3, un_val='ID_ech', view=t), gdf_viewer(df2, rows=3, un_val='ID_ech', view=t)

In [ ]:
df1.ID_ech = df1.ID_ech.astype(str)
df2.ID_ech = df2.ID_ech.astype(str)

In [ ]:
mdf, conflict_df=gdf_merger(df1, df2, how=how[1], on='ID_ech', date_col='Date_prv', dist_max=1)

#### Merge with object dataset

In [ ]:
dataset, conflict_df=gdf_merger(dataset, mdf, how=how[1], on='ID_ech', date_col='Date_prv', dist_max=1)

In [ ]:
if 'level_0' in dataset.columns:
    if 'index' in dataset.columns:
        dataset.drop(columns='index', inplace=True)
    dataset.rename(columns={'level_0':'index'}, inplace=True)

In [ ]:
gdf_viewer(dataset, rows=10, un_val='ID_ech', view=t)

#### $\color{blue}{\textbf{Manage conflicts}}$

In [ ]:
conflict_df

####  $\color{red}{\textbf{Save final object dataset}}$

In [ ]:
if 'index' in dataset.columns:
    dataset.drop(columns='index', inplace=True)

In [ ]:
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
    
dataset.to_csv(save_dir + save_file, index=False)

###  ------------------------------------- Testing area ------------------------------

#### $\color{green}{\textbf{Read and merge}}$

In [ ]:
a, b = 4, 0
file1= files_dict[key][a]
file2= files_dict[key][b]
print(files_dict[key][a].replace(work_dir,""),'||', files_dict[key][b].replace(work_dir,""))

In [ ]:
a, b = 4, 0
file1= files_dict[key][a]
file2= files_dict[key][b]
print(files_dict[key][a].replace(work_dir,""),'||', files_dict[key][b].replace(work_dir,""))

In [ ]:
#file1= work_dir + 'database_Memoris3/Donnees_piezos_Piezometers.csv' # 4
#file2= work_dir + 'Liste_XY/Sol_Eau_Piezometers.csv' # 0  

df1, df2 = create_df([file1, file2])
gdf_viewer(df1, rows=3, un_val='ID', view=t), gdf_viewer(df2, rows=3, un_val='ID', view=t)

In [ ]:
df1.ID_ech = df1.ID_ech.astype(str)
df2.ID_ech = df2.ID_ech.astype(str)

#### $\color{blue}{\textbf{Manage conflicts}}$

In [ ]:
file1= files_dict[key][0]
file2= files_dict[key][3]

df1, df2 = create_df([file1, file2])
print(files_dict[key][1].replace(work_dir,""),'||', files_dict[key][2].replace(work_dir,""))
gdf_viewer(df1, rows=3, un_val='ID', view=t), gdf_viewer(df2, rows=3, un_val='ID', view=t)

In [ ]:
fix_duplicates(df1, df2)

In [ ]:
gdf_viewer(df1, rows=3, un_val='ID', view=t), gdf_viewer(df2, rows=3, un_val='ID', view=t)

In [ ]:
df1.rename(columns={'Profondeur':'Long_for'}, inplace=True)
df2.rename(columns={'Profondeur':'Long_for'}, inplace=True)

In [ ]:
test, conf_test=gdf_merger(df1, df2, how=how[1], on='new_ID', dist_max=1)

In [ ]:
if len(conf_test) > 0:
    gdf_viewer(conf_test, rows=10, un_val='new_ID', view=t)

In [ ]:
gdf_viewer(test, rows=10, un_val='ID', view=t)

In [ ]:
pause

###  °°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°

# Measures

Some corrections todo in 'data organization':
- file 0 and file 1 are the same in result (check it)

In [ ]:
files_dict[key]

# Analysis

Some corrections todo in 'data organization':
- file 0 and file 1 are the same in result (check it)

In [ ]:
files_dict[key]

# Piezometers

In [ ]:
key='Piezometer'
save_file = f'Merged_Piezometers.csv'
coi=['ID','ID_date','X','Y','Z','Type','Long_for','Diam_for','Refus'] #columns of interest
dataset = pd.DataFrame()
print(len(files_dict[key]), 'files')

In [ ]:
files_dict[key]

In [ ]:
dataset_overview(files_dict[key])

#### $\color{green}{\textbf{Read and merge}}$

In [ ]:
file1= work_dir + 'Phase_1_Memoris/Result_eau_Piezometers.csv' # 2
file2= work_dir + 'Memoris_seafile/Result_eau_Piezometers.csv' # 1  


df1, df2 = create_df([file1, file2])
gdf_viewer(df1, rows=3, un_val='ID', view=t), gdf_viewer(df2, rows=3, un_val='ID', view=t)

In [ ]:
fix_duplicates(df1, df2, drop_old_id=True)

In [ ]:
mdf, conflict_df=gdf_merger(df1, df2, how=how[1], on='ID', dist_max=1)

#### First object dataset save

In [ ]:
dataset = mdf.copy() #saving

#### $\color{green}{\textbf{Read and merge}}$

In [ ]:
file1= work_dir + 'database_Memoris3/Donnees_piezos_Piezometers.csv' # 4
file2= work_dir + 'Liste_XY/Sol_Eau_Piezometers.csv' # 0  


df1, df2 = create_df([file1, file2])
gdf_viewer(df1, rows=3, un_val='ID', view=t), gdf_viewer(df2, rows=3, un_val='ID', view=t)

In [ ]:
df1, check = gdf_filter(df1, position=True, id_col='ID', expression='sup|prof', dist_max=1, drop=True, drop_old_id=True)

In [ ]:
df2, check = gdf_filter(df2, position=True, id_col='ID', expression='sup|prof', dist_max=1, drop=True)

In [ ]:
fix_duplicates(df1, df2)

In [ ]:
mdf, conflict_df=gdf_merger(df1, df2, how=how[1], on='ID', dist_max=1)

#### $\color{blue}{\textbf{Manage conflicts}}$

In [ ]:
conflict_df

In [ ]:
data_validation(overall_data=mdf, conflict_data=conflict_df, index_col='index', 
                valid_dict={'Nappe_y':list(conflict_df.index)[:18]})

In [ ]:
conflict_df

#### Merge with object dataset

In [ ]:
dataset, conflict_df=gdf_merger(dataset, mdf, how=how[1], on='ID', dist_max=1)

In [ ]:
if 'level_0' in dataset.columns:
    if 'index' in dataset.columns:
        dataset.drop(columns='index', inplace=True)
    dataset.rename(columns={'level_0':'index'}, inplace=True)

In [ ]:
gdf_viewer(dataset, rows=10, un_val='ID', view=t)

#### $\color{green}{\textbf{Read and merge}}$

In [ ]:
a, b = 3, 5
file1= files_dict[key][a]
file2= files_dict[key][b]
print(files_dict[key][a].replace(work_dir,""),'||', files_dict[key][b].replace(work_dir,""))

In [ ]:
file1= work_dir + 'Phase_2_Memoris/Result_eau_Piezometers.csv' # 3
file2= work_dir + 'database_Memoris3/Drains_Pz_ENEL_Piezometers.csv' # 5  


df1, df2 = create_df([file1, file2])
gdf_viewer(df1, rows=3, un_val='ID', view=t), gdf_viewer(df2, rows=3, un_val='ID', view=t)

In [ ]:
mdf, conflict_df=gdf_merger(df1, df2, how=how[1], on='ID', dist_max=1)

#### Merge with object dataset

In [ ]:
dataset, conflict_df=gdf_merger(dataset, mdf, how=how[1], on='ID', dist_max=1)

In [ ]:
if 'level_0' in dataset.columns:
    if 'index' in dataset.columns:
        dataset.drop(columns='index', inplace=True)
    dataset.rename(columns={'level_0':'index'}, inplace=True)

In [ ]:
gdf_viewer(dataset, rows=10, un_val='ID', view=t)

#### $\color{green}{\textbf{Read and merge}}$

In [ ]:
file1= work_dir + 'donnees_terrain_2019/Donnees_forage_Piezometers.csv' # 9
file2= work_dir + 'database_Memoris3/Result_eau_Piezometers.csv' # 7  

df1, df2 = create_df([file1, file2])
gdf_viewer(df1, rows=3, un_val='ID', view=t), gdf_viewer(df2, rows=3, un_val='ID', view=t)

In [ ]:
df1.ID = df1.ID.astype(str)
df2.ID = df2.ID.astype(str)

In [ ]:
df2, check = gdf_filter(df2, position=True, id_col='ID', expression='sup|prof', dist_max=1, val_max=1.1, drop=True, drop_old_id=True)

In [ ]:
gdf_viewer(df2, rows=10, un_val='ID', view=t)

In [ ]:
fix_duplicates(df1, df2)

In [ ]:
mdf, conflict_df=gdf_merger(df1, df2, how=how[1], on='ID', dist_max=1)

#### Merge with object dataset

In [ ]:
dataset, conflict_df=gdf_merger(dataset, mdf, how=how[1], on='ID', dist_max=1)

#### $\color{blue}{\textbf{Manage conflicts}}$

In [ ]:
conflict_df

In [ ]:
data_validation(overall_data=mdf, conflict_data=conflict_df, index_col='index', 
                valid_dict={'Nappe_y':list(conflict_df.index)[:18]})

In [ ]:
conflict_df

In [ ]:
if 'level_0' in dataset.columns:
    if 'index' in dataset.columns:
        dataset.drop(columns='index', inplace=True)
    dataset.rename(columns={'level_0':'index'}, inplace=True)

In [ ]:
gdf_viewer(dataset, rows=10, un_val='ID', view=t)

#### $\color{green}{\textbf{Read and merge}}$

In [ ]:
a, b = 13, 15
file1= files_dict[key][a]
file2= files_dict[key][b]
print(files_dict[key][a].replace(work_dir,""),'||', files_dict[key][b].replace(work_dir,""))

In [ ]:
#file1= work_dir + 'donnees_terrain_2019/Donnees_forage_Piezometers.csv' # 9
#file2= work_dir + 'database_Memoris3/Result_eau_Piezometers.csv' # 7  

df1, df2 = create_df([file1, file2])
gdf_viewer(df1, rows=3, un_val='ID', view=t), gdf_viewer(df2, rows=3, un_val='ID', view=t)

In [ ]:
fix_duplicates(df1, df2)

In [ ]:
mdf, conflict_df=gdf_merger(df1, df2, how=how[1], on='ID', dist_max=1)

#### Merge with object dataset

In [ ]:
dataset, conflict_df=gdf_merger(dataset, mdf, how=how[1], on='ID', dist_max=1)

#### $\color{blue}{\textbf{Manage conflicts}}$

In [ ]:
conflict_df # i think there are not the same, but no date or postition to distinguish them !
# --> check boreholes sheets (pdf)

In [ ]:
gdf_viewer(dataset, rows=3, un_val='ID', view=t)

####  $\color{red}{\textbf{Save final object dataset}}$

In [ ]:
if 'index' in piezometers.columns:
    piezometers.drop(columns='index', inplace=True)

In [ ]:
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
    
piezometers.to_csv(save_dir + save_file, index=False)

###  ------------------------------------- Testing area ------------------------------

#### $\color{green}{\textbf{Read and merge}}$

In [ ]:
a, b = 4, 0
file1= files_dict[key][a]
file2= files_dict[key][b]
print(files_dict[key][a].replace(work_dir,""),'||', files_dict[key][b].replace(work_dir,""))
#file1= work_dir + 'database_Memoris3/Donnees_piezos_Piezometers.csv' # 4
#file2= work_dir + 'Liste_XY/Sol_Eau_Piezometers.csv' # 0  


df1, df2 = create_df([file1, file2])
gdf_viewer(df1, rows=3, un_val='ID', view=t), gdf_viewer(df2, rows=3, un_val='ID', view=t)

#### $\color{blue}{\textbf{Manage conflicts}}$

In [ ]:
file1= files_dict[key][0]
file2= files_dict[key][3]

df1, df2 = create_df([file1, file2])
print(files_dict[key][1].replace(work_dir,""),'||', files_dict[key][2].replace(work_dir,""))
gdf_viewer(df1, rows=3, un_val='ID', view=t), gdf_viewer(df2, rows=3, un_val='ID', view=t)

In [ ]:
fix_duplicates(df1, df2)

In [ ]:
gdf_viewer(df1, rows=3, un_val='ID', view=t), gdf_viewer(df2, rows=3, un_val='ID', view=t)

In [ ]:
df1.rename(columns={'Profondeur':'Long_for'}, inplace=True)
df2.rename(columns={'Profondeur':'Long_for'}, inplace=True)

In [ ]:
test, conf_test=gdf_merger(df1, df2, how=how[1], on='new_ID', dist_max=1)

In [ ]:
if len(conf_test) > 0:
    gdf_viewer(conf_test, rows=10, un_val='new_ID', view=t)

In [ ]:
gdf_viewer(test, rows=10, un_val='ID', view=t)

In [ ]:
pause

###  °°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°

In [ ]:
file1= files_dict[key][6]
file2= files_dict[key][4]

df1, df2 = create_df([file1, file2])
gdf_viewer(df1, rows=3, un_val='ID', view=t), gdf_viewer(df2, rows=3, un_val='ID', view=t)

In [ ]:
file1= files_dict[key][0]
file2= files_dict[key][1]

df1, df2 = create_df([file1, file2])
gdf_viewer(df1, rows=3, un_val='ID', view=t), gdf_viewer(df2, rows=3, un_val='ID', view=t)

In [ ]:
mdf, conflict_df=gdf_merger(df1, df2, how=how[1], on='ID', dist_max=1)
check_col(mdf)

##### check and validate duplicate objects
- The function "gdf_filter()" doesn't work in some cases, so we use function "doubled_objects_check()"
- we have same objects Names but differents by positions here

In [ ]:
mdf, check = gdf_filter(mdf, position=True, id_on='ID', expression='sup|prof', dist_max=1, drop=True, rapp_val=1)

In [ ]:
double_objects_check(mdf)

In [ ]:
drop_id = [2,25,30] # objects are seemingly the same, but is it possible to get 2 objects so close (~ 1m)?
mdf.drop(index=drop_id, inplace=True)
mdf.reset_index(drop=True, inplace=True)

In [ ]:
gdf_viewer(mdf, rows=3, cols=13, un_val='ID', view=t)

#### Merge with object type dataset

In [ ]:
piezometers = mdf.copy() #saving

In [ ]:
file1= files_dict[key][2]
file2= files_dict[key][3]

df1, df2 = create_df([file1, file2])
gdf_viewer(df1, rows=3, un_val='ID', view=t), gdf_viewer(df2, rows=3, un_val='ID', view=t)

In [ ]:
mdf, conflict_df=gdf_merger(df1, df2, how=how[1], on='ID', dist_max=2)
check_col(mdf)

In [ ]:
gdf_viewer(mdf, rows=3, cols=13, un_val='ID', view=f)

In [ ]:
dataset, conflict_df=gdf_merger(dataset, mdf, how=how[1], on='ID', dist_max=1)
check_col(mdf)

In [ ]:
gdf_viewer(dataset, rows=3, cols=13, un_val='ID', view=t)

In [ ]:
double_objects_check(piezometers)

In [ ]:
drop_id = [292, 293]
piezometers.drop(index=drop_id, inplace=True)
gdf_viewer(dataset, rows=5, un_val='ID', view=f)

In [ ]:
file1= files_dict[key][4]
file2= files_dict[key][5]

df1, df2 = create_df([file1, file2])
gdf_viewer(df1, rows=3, un_val='ID', view=t), gdf_viewer(df2, rows=3, un_val='ID', view=t)

In [ ]:
mdf, conflict_df=gdf_merger(df1, df2, how=how[1], on='ID', dist_max=2)
check_col(mdf)

In [ ]:
dataset, conflict_df=gdf_merger(dataset, mdf, how=how[1], on='ID', dist_max=1)
check_col(mdf)

In [ ]:
gdf_viewer(dataset, rows=3, cols=13, un_val='ID', view=t)

In [ ]:
dataset, check = gdf_filter(dataset, position=True, id_on='ID', expression='sup|prof', dist_max=1, drop=True)
#gdf_viewer(dataset, rows=5, un_val='ID', view=t)

In [ ]:
double_objects_check(piezometers)

In [ ]:
drop_id = [2,4,30,94,106]
piezometers.drop(index=drop_id, inplace=True)
gdf_viewer(dataset, rows=5, un_val='ID', view=f)

In [ ]:
file1= files_dict[key][6]
file2= files_dict[key][9]

df1, df2 = create_df([file1, file2])
gdf_viewer(df1, rows=3, un_val='ID', view=t), gdf_viewer(df2, rows=3, un_val='ID', view=t)

In [ ]:
df2['ID'] = df2.ID.astype('object')

In [ ]:
mdf, conflict_df=gdf_merger(df1, df2, how=how[1], on='ID', dist_max=2)
check_col(mdf)

In [ ]:
dataset, conflict_df=gdf_merger(dataset, mdf, how=how[1], on='ID', dist_max=1)
check_col(mdf)

In [ ]:
gdf_viewer(dataset, rows=3, cols=13, un_val='ID', view=t)

In [ ]:
file1= files_dict[key][10]
file2= files_dict[key][11]

df1, df2 = create_df([file1, file2])
gdf_viewer(df1, rows=3, un_val='ID', view=t), gdf_viewer(df2, rows=3, un_val='ID', view=t)

In [ ]:
df1['ID'] = df1.ID.astype('object')

In [ ]:
mdf, conflict_df=gdf_merger(df1, df2, how=how[1], on='ID', dist_max=2)
check_col(mdf)

In [ ]:
dataset, conflict_df=gdf_merger(dataset, mdf, how=how[1], on='ID', dist_max=1)
check_col(mdf)

In [ ]:
conflict_df

In [ ]:
gdf_viewer(dataset, rows=3, cols=13, un_val='ID', view=t)

In [ ]:
file1= files_dict[key][12]
file2= files_dict[key][13]

df1, df2 = create_df([file1, file2])
gdf_viewer(df1, rows=3, un_val='ID', view=t), gdf_viewer(df2, rows=3, un_val='ID', view=t)

In [ ]:
mdf, conflict_df=gdf_merger(df1, df2, how=how[1], on='ID', dist_max=2)
check_col(mdf)

In [ ]:
conflict_df

In [ ]:
dataset, conflict_df=gdf_merger(dataset, mdf, how=how[1], on='ID', dist_max=1)
check_col(mdf)

In [ ]:
gdf_viewer(dataset, rows=3, cols=13, un_val='ID', view=t)

In [ ]:
file1= files_dict[key][14]
file2= files_dict[key][15]

df1, df2 = create_df([file1, file2])
gdf_viewer(df1, rows=3, un_val='ID', view=t), gdf_viewer(df2, rows=3, un_val='ID', view=t)

In [ ]:
mdf, conflict_df=gdf_merger(df1, df2, how=how[1], on='ID', dist_max=2)
check_col(mdf)

In [ ]:
dataset, conflict_df=gdf_merger(dataset, mdf, how=how[1], on='ID', dist_max=1)
check_col(mdf)

In [ ]:
conflict_df

In [ ]:
gdf_viewer(dataset, rows=3, cols=13, un_val='ID', view=t)

In [ ]:
file1= files_dict[key][16]
df1 = pd.read_csv(file1, delimiter=',')

print(f"df1 : {file1.replace(work_dir,'')}")
gdf_viewer(df1, rows=3, un_val='ID', view=t)

#### Last merging

In [ ]:
dataset, conflict_df=gdf_merger(dataset, df1, how=how[1], on='ID', dist_max=1)
check_col(mdf)

In [ ]:
conflict_df

In [ ]:
gdf_viewer(dataset, rows=3, cols=13, un_val='ID', view=t)

####  $\color{red}{\textbf{Save final Piezometers data}}$

In [ ]:
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
    
piezometers.to_csv(save_dir+save_file, index=False)

==========================================================================================================

# Lithologies

Do not add parameter 'dist_max' when merging without considering position !!! otherwise, unuseless rows added

In [ ]:
key='Litho'
save_file = f'Merged_Lithologies.csv'
coi=['ID','X','Y','Z','Litho_top','Litho_base','Description']  #columns of interest
lithologies = pd.DataFrame()
print(len(files_dict[key]), 'files')

In [ ]:
files_dict[key]

In [ ]:
file1= files_dict[key][0]
file2= files_dict[key][3]

df1, df2 = create_df([file1, file2])
gdf_viewer(df1, rows=3, un_val='ID', view=t), gdf_viewer(df2, rows=3, un_val='ID', view=t)

In [ ]:
mdf, conflict_df=gdf_merger(df1, df2, how=how[1], on='ID')#, step_merge
check_col(mdf)

In [ ]:
gdf_viewer(mdf, rows=3, cols=13, un_val='ID', view=t)

In [ ]:
gdf_viewer(conflict_df, rows=5, un_val='ID', view=t) #conflict_df.ID.unique()

In [ ]:
common_cols = list(set(df1.columns) & set(df2.columns))
test1 = df1.merge(df2, how = 'inner', on='ID')
test2 = df1.merge(df2, how = 'outer', on='ID', indicator=True).loc[lambda x : x.query('_merge =="right_only" or _merge=="left_only"').index]
test3 = test1.merge(test2, how = 'outer', on='ID')
test4 = df1.merge(df2, how = 'outer', on=list(common_cols))
print((len(test1), len(test2), len(test3)))
gdf_viewer(test4)

In [ ]:
lithologies = mdf.copy() #saving

In [ ]:
file1= files_dict[key][2]
file2= files_dict[key][4]

df1, df2 = create_df([file1, file2])
gdf_viewer(df1, rows=3, un_val='ID', view=t), gdf_viewer(df2, rows=3, un_val='ID', view=t)

In [ ]:
mdf, conflict_df=gdf_merger(df1, df2, how=how[1], on='ID')
check_col(mdf)

In [ ]:
gdf_viewer(mdf, rows=3, cols=13, un_val='ID', view=t)

##### Lithologies merging 

In [ ]:
lithologies, conflict_df=gdf_merger(lithologies, mdf, how=how[1], on='ID')
check_col(mdf)

In [ ]:
gdf_viewer(lithologies, rows=10, cols=15, un_val='ID', view=t)

In [ ]:
file1= files_dict[key][5]
file2= files_dict[key][6]

df1, df2 = create_df([file1, file2])
gdf_viewer(df1, rows=3, un_val='ID', view=t), gdf_viewer(df2, rows=3, un_val='ID', view=t)

In [ ]:
gdf_viewer(df1.merge(df2, how='inner', on='ID'), rows=5, cols=15, un_val='ID', view=t)

In [ ]:
gdf_viewer(df1.merge(df2, how = 'outer', on='ID',indicator=True), rows=5, cols=15, un_val='ID', view=t)

In [ ]:
gdf_viewer(mdf, rows=3, cols=13, un_val='ID', view=t)

In [ ]:
mdf, conflict_df=gdf_merger(df1, df2, how=how[1], on='ID', dist_max=1)
check_col(mdf)

In [ ]:
gdf_viewer(conflict_df, rows=5, un_val='ID', view=t) #conflict_df.ID.unique()

##### Lithologies merging 

In [ ]:
lithologies, conflict_df=gdf_merger(lithologies, mdf, how=how[1], on='ID', dist_max=1)
check_col(mdf)

In [ ]:
gdf_viewer(lithologies, rows=3, un_val='ID', view=t)

In [ ]:
stop